# Continuous-Bag-of-Words (CBOW) Eembeddings
This file achieves word embedding based on the word2vec paradigm.

## Download \& Preprocess the Data

In [1]:
!pip install torchdata==0.5.1
!pip install torchtext==0.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.7.0
    Uninstalling torchdata-0.7.0:
      Successfully uninstalled torchdata-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torcha

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if __name__=='__main__':
    print('Using device:', DEVICE)

Using device: cuda


In [3]:
# Constants (feel free to use these in your code, but do not change them)
CBOW_START = "<s>"   # Start-of-sentence token
CBOW_END = "</s>"    # End-of-sentence-token
CBOW_UNK = "<UNK>"   # Unknown word token

In [4]:
import torchtext
import random
import sys

def cbow_preprocess(data, vocab=None, do_lowercase=True):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else CBOW_UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else CBOW_UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = {}
    for i in range(len(final_data)):
        # Make words lowercase for this assignment
        final_data[i] = [x.lower() if do_lowercase and x != CBOW_UNK else x for x in final_data[i]]
        final_data[i] = [CBOW_START] + final_data[i] + [CBOW_END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab[word] = vocab.get(word, 0) + 1
    return final_data, vocab

def getDataset():
    dataset = torchtext.datasets.WikiText2(root='.data', split=('train',))
    train_dataset, vocab = cbow_preprocess(dataset[0])
    return train_dataset, vocab

if __name__=='__main__':
    sentences, vocab = getDataset()

In [5]:
if __name__ == '__main__':
    for x in random.sample(sentences, 10):
        print (x)

['<s>', 'tech', 'began', 'the', 'second', 'quarter', 'in', 'possession', 'of', 'the', 'ball', 'with', 'a', 'first', 'down', 'at', 'their', '13', '@-@', 'yard', 'line', '.', '</s>']
['<s>', 'the', 'design', 'by', 'john', 'pemberton', 'won', 'a', 'design', 'council', 'award', ',', 'and', 'there', 'are', 'examples', 'of', 'the', 'sovereign', 'in', 'the', 'museum', 'of', 'modern', 'art', 'in', 'new', 'york', '.', '</s>']
['<s>', 'as', 'regards', 'the', 'music', 'of', 'his', 'own', 'time', ',', 'alkan', 'was', '<UNK>', ',', 'or', 'at', 'any', 'rate', 'detached', '.', '</s>']
['<s>', 'cougars', 'have', 'large', 'paws', 'and', 'proportionally', 'the', 'largest', 'hind', 'legs', 'in', 'the', 'cat', 'family', '.', '</s>']
['<s>', 'its', 'diameter', 'is', 'approximately', '<UNK>', 'kilometers', '(', '<UNK>', 'miles', ')', ',', 'making', 'it', 'the', 'largest', 'of', 'the', 'minor', 'planets', 'within', 'the', 'orbit', 'of', 'neptune', '.', '</s>']
['<s>', 'the', 'democratic', 'primary', 'was', '

## Define the dataset class

In [6]:
from torch.utils import data
from collections import defaultdict

class CbowDataset(data.Dataset):
    def __init__(self, sentences, vocab, context_size):

        assert CBOW_START in vocab and CBOW_END in vocab and CBOW_UNK in vocab
        self.sentences = sentences
        self.context_size = context_size

        self.training_examples = []
        self.make_training_examples()

        self.word2idx = {} # Mapping of word to index
        self.idx2word = {} # Mapping of index to word
        self.build_dictionaries(sorted(vocab.keys()))
        self.vocab_size = len(self.word2idx)

    def make_training_examples(self):
        '''
        Builds a list of context-target_word pairs that will be used as training examples for the model and stores them in
        self.training_examples.
        Each example is a (context, target_word) tuple, where context is a list of strings of size 2*context_size and
        target_word is simply a string.
        Returns nothing.
        '''

        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                start = max(0, i - self.context_size) # index of the start of the word
                end = min(len(sentence), i + self.context_size + 1) # index of the end of the word
                context = sentence[start:i] + sentence[i+1:end]
                if len(context) == 2 * self.context_size: # check if the list is made up of 2c words
                    self.training_examples.append((context, word))

    def build_dictionaries(self, vocab):
        '''
        Builds the dictionaries self.idx2word and self.word2idx. Make sure that you assign indices
        in the order the words appear in vocab (a list of words).
        Returns nothing.
        '''

        for i, word in enumerate(vocab):
            self.idx2word[i] = word
            self.word2idx[word] = i

    def get_context_vector(self, idx):
        '''
        Returns the context vector (as a torch.tensor) for the training example at index idx.
        This is is a tensor containing the indices of each word in the context.
        '''

        assert len(self.training_examples) > 0
        context_words = self.training_examples[idx][0]
        context_indices = [self.word2idx[word] for word in context_words]
        return torch.tensor(context_indices)

    def get_target_index(self, idx):
        '''
        Returns the index of the target word (as type int) of the training example at index idx.
        '''

        traget_word = self.training_examples[idx][1]
        return self.word2idx[traget_word]

    def __len__(self):
        '''
        Returns the number of training examples (as type int) in the dataset
        '''

        return len(self.training_examples)

    def __getitem__(self, idx):
        '''
        Returns the context vector (as a torch.tensor) and target index (as type int) of the training example at index idx.
        '''

        context_vector = self.get_context_vector(idx)
        traget_index = self.get_target_index(idx)
        return context_vector, traget_index

Sanity check


In [7]:

def sanityCheckCbowDataset():
    #	Read in the sample corpus
    test_sents = [['<s>', 'the', 'man', 'walks', 'the', 'dog', 'in', 'the', 'park', '</s>'],
            ['<s>', 'i', 'saw', 'the', 'man', 'with', 'the', 'telescope', 'on', 'the', CBOW_UNK, '</s>']]
    test_vocab = {'<s>':2, 'the':6, 'man':2, 'walks': 1, 'dog': 1, 'in': 1, 'park': 1, 'i': 1, 'saw': 1, 'with': 1, 'telescope':1, 'on': 1, CBOW_UNK: 1, '</s>': 2}
    print("Sample dataset:")
    for x in test_sents: print(x)

    context_sizes = [1,3,5]

    print('\n--- TEST: training_examples ---')
    training_examples_expected = [[(['<s>', 'man'], 'the'), (['the', 'walks'], 'man'), (['man', 'the'], 'walks'), (['walks', 'dog'], 'the'), (['the', 'in'], 'dog'), (['dog', 'the'], 'in'), (['in', 'park'], 'the'), (['the', '</s>'], 'park'), (['<s>', 'saw'], 'i'), (['i', 'the'], 'saw'), (['saw', 'man'], 'the'), (['the', 'with'], 'man'), (['man', 'the'], 'with'), (['with', 'telescope'], 'the'), (['the', 'on'], 'telescope'), (['telescope', 'the'], 'on'), (['on', '<UNK>'], 'the'), (['the', '</s>'], '<UNK>')],
                                  [(['<s>', 'the', 'man', 'the', 'dog', 'in'], 'walks'), (['the', 'man', 'walks', 'dog', 'in', 'the'], 'the'), (['man', 'walks', 'the', 'in', 'the', 'park'], 'dog'), (['walks', 'the', 'dog', 'the', 'park', '</s>'], 'in'), (['<s>', 'i', 'saw', 'man', 'with', 'the'], 'the'), (['i', 'saw', 'the', 'with', 'the', 'telescope'], 'man'), (['saw', 'the', 'man', 'the', 'telescope', 'on'], 'with'), (['the', 'man', 'with', 'telescope', 'on', 'the'], 'the'), (['man', 'with', 'the', 'on', 'the', '<UNK>'], 'telescope'), (['with', 'the', 'telescope', 'the', '<UNK>', '</s>'], 'on')],
                                  [(['<s>', 'i', 'saw', 'the', 'man', 'the', 'telescope', 'on', 'the', '<UNK>'], 'with'), (['i', 'saw', 'the', 'man', 'with', 'telescope', 'on', 'the', '<UNK>', '</s>'], 'the')]
                                 ]
    for i in range(len(context_sizes)):
        c=context_sizes[i]
        test_dataset = CbowDataset(test_sents, test_vocab, c)
        has_passed, message = True, ''
        training_examples = test_dataset.training_examples
        expected = training_examples_expected[i]
        if has_passed and len(training_examples) != len(expected):
            has_passed, message = False, 'len(training_examples) is incorrect. Expected: ' + str(len(expected)) + '\tGot: ' + str(len(training_examples))
        if has_passed and set([(type(x), len(x)) for x in training_examples]) != {(tuple, 2)}:
            has_passed, message = False, 'Each item of training_examples must be a 2-tuple; at least one of your items is not a 2-tuple.'
        if has_passed and set([(type(x[0]), type(x[1])) for x in training_examples]) != {(list, str)}:
            has_passed, message = False, 'Each item must contain a list of context words and a target word as a string. At least one of your items does not meet this condition.'
        if has_passed and sorted(training_examples, key = lambda x: (' '.join(x[0]), x[1])) != sorted(expected, key = lambda x: (' '.join(x[0]), x[1])):
            has_passed, message = False, 'training_examples is incorrect (note that the order of the examples does not matter). Expected: '+str(sorted(expected, key = lambda x: (' '.join(x[0]), x[1]))) + '\tGot: ' + str(sorted(training_examples, key = lambda x: (' '.join(x[0]), x[1])))

        status = 'PASSED' if has_passed else 'FAILED'
        print('\tcontext_size:', c, '\t'+status, '\t'+message)

    print('\n--- TEST: idx2word and word2idx dictionaries ---')
    expected_word2idx = {'</s>': 0, '<UNK>': 1, '<s>': 2, 'dog': 3, 'i': 4, 'in': 5, 'man': 6, 'on': 7, 'park': 8, 'saw': 9, 'telescope': 10, 'the': 11, 'walks': 12, 'with': 13}
    expected_idx2word = {0: '</s>', 1: '<UNK>', 2: '<s>', 3: 'dog', 4: 'i', 5: 'in', 6: 'man', 7: 'on', 8: 'park', 9: 'saw', 10: 'telescope', 11: 'the', 12: 'walks', 13: 'with'}
    for i in range(len(context_sizes)):
        c=context_sizes[i]
        test_dataset = CbowDataset(test_sents, test_vocab, c)
        has_passed, message = True, ''
        word2idx = test_dataset.word2idx
        idx2word = test_dataset.idx2word

        has_passed, message = True, ''
        if has_passed and (test_dataset.vocab_size != len(test_dataset.word2idx) or test_dataset.vocab_size != len(test_dataset.idx2word)):
            has_passed, message = False, 'dataset.vocab_size (' + str(test_dataset.vocab_size) + ') must be the same length as dataset.word2idx (' + str(len(test_dataset.word2idx)) + ') and dataset.idx2word ('+str(len(test_dataset.idx2word)) +').'
        if has_passed and (test_dataset.vocab_size != len(expected_word2idx)):
            has_passed, message = False, 'Your vocab size is incorrect. Expected: ' + str(len(expected_word2idx)) + '\tGot: ' + str(test_dataset.vocab_size)
        if has_passed and sorted(list(test_dataset.idx2word.keys())) != list(range(0, test_dataset.vocab_size)):
            has_passed, message = False, 'dataset.idx2word must have keys ranging from 0 to dataset.vocab_size-1. Keys in your dataset.idx2word: ' + str(sorted(list(test_dataset.idx2word.keys())))
        if has_passed and sorted(list(test_dataset.word2idx.keys())) != sorted(list(expected_word2idx.keys())):
            has_passed, message = False, 'Your dataset.word2idx has incorrect keys. Expected: ' + str(sorted(list(expected_word2idx.keys()))) + '\tGot: ' + str(sorted(list(test_dataset.word2idx.keys())))
        if has_passed: # Check that word2idx and idx2word are consistent
            widx = sorted(list(test_dataset.word2idx.items()))
            idxw = sorted(list([(v,k) for k,v in test_dataset.idx2word.items()]))
            if not (len(widx) == len(idxw) and all([widx[q] == idxw[q] for q in range(len(widx))])):
                has_passed, message = False, 'Your dataset.word2idx and dataset.idx2word are not consistent. dataset.idx2word: ' + str(test_dataset.idx2word) + '\tdataset.word2idx: ' + str(test_dataset.word2idx)
        if has_passed and word2idx != expected_word2idx:
            has_passed, message = False, 'Your dataset.word2idx is incorrect. Expected: ' + str(expected_word2idx) + '\tGot: ' + str(word2idx)
        if has_passed and idx2word != expected_idx2word:
            has_passed, message = False, 'Your dataset.word2idx is incorrect. Expected: ' + str(expected_idx2word) + '\tGot: ' + str(idx2word)

        status = 'PASSED' if has_passed else 'FAILED'
        print('\tcontext_size:', c, '\t'+status, '\t'+message)

    print('\n--- TEST: len(dataset) ---')
    correct_lens = [18,10,2]
    for i in range(len(context_sizes)):
        c=context_sizes[i]
        test_dataset = CbowDataset(test_sents, test_vocab, c)
        has_passed = len(test_dataset) == correct_lens[i]
        status = 'PASSED' if has_passed else 'FAILED'
        if has_passed: message = ''
        else: message = 'len(dataset) is incorrect. Expected: ' + str(correct_lens[i]) + '\tGot: ' + str(len(test_dataset))
        print('\tcontext_size:', c, '\t'+status, '\t'+message)

    print('\n--- TEST: __getitem__(self, idx) ---')
    for i in range(len(context_sizes)):
        c=context_sizes[i]
        test_dataset = CbowDataset(test_sents, test_vocab, c)
        for j in range(correct_lens[i]):
            returned = test_dataset.__getitem__(j)

            has_passed, message = True, ''
            if has_passed and len(returned) != 2:
                has_passed, message = False, 'dataset.__getitem__(idx) must return 2 items. Got ' + str(len(returned)) +' items instead.'
            if has_passed and (type(returned[0]) != torch.Tensor or type(returned[1]) != int):
                has_passed, message = False, 'The context vector must be a torch.Tensor and the target index must be an int. Got: (' + str(type(returned[0])) + ', ' + str(type(returned[1])) + ')'
            if has_passed and (returned[0].shape != torch.randint(0,100,(2*c,)).shape):
                has_passed, message = False, 'Shape of first return is incorrect. Expected: ' + str(torch.randint(0,100,(2*c,)).shape) + '.\tGot: ' + str(returned[0].shape)

            status = 'PASSED' if has_passed else 'FAILED'
            print('\tcontext_size:', c, '\tidx:',str(j),'\t' if j<10 else '','\t',status, '\t'+message)

if __name__ == '__main__':
    sanityCheckCbowDataset()

Sample dataset:
['<s>', 'the', 'man', 'walks', 'the', 'dog', 'in', 'the', 'park', '</s>']
['<s>', 'i', 'saw', 'the', 'man', 'with', 'the', 'telescope', 'on', 'the', '<UNK>', '</s>']

--- TEST: training_examples ---
	context_size: 1 	PASSED 	
	context_size: 3 	PASSED 	
	context_size: 5 	PASSED 	

--- TEST: idx2word and word2idx dictionaries ---
	context_size: 1 	PASSED 	
	context_size: 3 	PASSED 	
	context_size: 5 	PASSED 	

--- TEST: len(dataset) ---
	context_size: 1 	PASSED 	
	context_size: 3 	PASSED 	
	context_size: 5 	PASSED 	

--- TEST: __getitem__(self, idx) ---
	context_size: 1 	idx: 0 	 	 PASSED 	
	context_size: 1 	idx: 1 	 	 PASSED 	
	context_size: 1 	idx: 2 	 	 PASSED 	
	context_size: 1 	idx: 3 	 	 PASSED 	
	context_size: 1 	idx: 4 	 	 PASSED 	
	context_size: 1 	idx: 5 	 	 PASSED 	
	context_size: 1 	idx: 6 	 	 PASSED 	
	context_size: 1 	idx: 7 	 	 PASSED 	
	context_size: 1 	idx: 8 	 	 PASSED 	
	context_size: 1 	idx: 9 	 	 PASSED 	
	context_size: 1 	idx: 10  	 PASSED 	
	context

## Define the CBOW Model

In [8]:
class CbowModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, context_size):
        '''
        vocab_size: Size of the vocabulary
        embed_size: Size of your embedding vectors
        hidden_size: Size of hidden layer of neural network
        context_size: The size of your context window used to generate training examples
        '''
        super(CbowModel, self).__init__()

        self.context_size = context_size

        self.embedding = nn.Embedding(vocab_size, embed_size)

        self.linear1 = nn.Linear(embed_size, hidden_size)

        self.linear2 = nn.Linear(hidden_size, vocab_size)

    def forward(self, inputs):
        '''
        inputs: Tensor of size [batch_size, 2*context_size]

        Returns output: Tensor of size [batch_size, vocab_size]
        '''

        embeddings = self.embedding(inputs)

        avarge_embeddings = torch.mean(embeddings, 1)

        linear_out = F.relu(self.linear1(avarge_embeddings))

        linear_out = self.linear2(linear_out)

        return linear_out

Sanity Check

In [9]:
count_parameters = lambda model: sum(p.numel() for p in model.parameters() if p.requires_grad)

def makeCbowSanityBatch(test_params):
    batch_size = test_params['batch_size']
    new_test_params = {k:v for k,v in test_params.items() if k != 'batch_size'}
    batch = torch.randint(0, new_test_params['vocab_size'], (batch_size,new_test_params['context_size']*2))
    return batch, new_test_params

def sanityCheckModel(all_test_params, NN, expected_outputs, init_or_forward, make_batch_fxn=None):
    print('--- TEST: ' + ('Number of Model Parameters (tests __init__(...))' if init_or_forward=='init' else 'Output shape of forward(...)') + ' ---')

    for tp_idx, (test_params, expected_output) in enumerate(zip(all_test_params, expected_outputs)):
        if init_or_forward == "forward":
            input, test_params = make_batch_fxn(test_params)

        # Construct the student model
        tps = {k:v for k, v in test_params.items()}
        stu_nn = NN(**tps)

        if init_or_forward == "forward":
            with torch.no_grad():
                stu_out = stu_nn(input)
            ref_out_shape = expected_output

            has_passed = torch.is_tensor(stu_out)
            if not has_passed: msg = 'Output must be a torch.Tensor; received ' + str(type(stu_out))
            else:
                has_passed = stu_out.shape == ref_out_shape
                msg = 'Your Output Shape: ' + str(stu_out.shape)


            status = 'PASSED' if has_passed else 'FAILED'
            message = '\t' + status + "\t Init Input: " + str({k:v for k,v in tps.items()}) + '\tForward Input Shape: ' + str(input.shape) + '\tExpected Output Shape: ' + str(ref_out_shape) + '\t' + msg
            print(message)
        else:
            stu_num_params = count_parameters(stu_nn)
            ref_num_params = expected_output
            comparison_result = (stu_num_params == ref_num_params)

            status = 'PASSED' if comparison_result else 'FAILED'
            message = '\t' + status + "\tInput: " + str({k:v for k,v in test_params.items()}) + ('\tExpected Num. Params: ' + str(ref_num_params) + '\tYour Num. Params: '+ str(stu_num_params))
            print(message)

        del stu_nn


if __name__ == '__main__':
    # Test init
    cbow_init_inputs = [{'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2}, {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4}, {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 128, 'context_size': 2}, {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 128, 'context_size': 4}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 128, 'context_size': 2}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 128, 'context_size': 4}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 128, 'context_size': 2}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 128, 'context_size': 4}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 128, 'context_size': 2}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 128, 'context_size': 4}]
    cbow_init_expected_outputs = [3082, 3082, 5834, 5834, 5450, 5450, 10250, 10250, 99112, 99112, 165224, 165224, 133160, 133160, 201320, 201320]

    sanityCheckModel(cbow_init_inputs, CbowModel, cbow_init_expected_outputs, "init")
    print()

    # Test forward
    cbow_forward_inputs = [{'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2, 'batch_size': 1}, {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2, 'batch_size': 5}, {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2, 'batch_size': 500}, {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4, 'batch_size': 1}, {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4, 'batch_size': 5}, {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4, 'batch_size': 500}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2, 'batch_size': 1}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2, 'batch_size': 5}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2, 'batch_size': 500}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4, 'batch_size': 1}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4, 'batch_size': 5}, {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4, 'batch_size': 500}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2, 'batch_size': 1}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2, 'batch_size': 5}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2, 'batch_size': 500}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4, 'batch_size': 1}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4, 'batch_size': 5}, {'vocab_size': 1000, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4, 'batch_size': 500}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2, 'batch_size': 1}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2, 'batch_size': 5}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2, 'batch_size': 500}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4, 'batch_size': 1}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4, 'batch_size': 5}, {'vocab_size': 1000, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4, 'batch_size': 500}]
    cbow_forward_expected_outputs = [torch.Size([1, 10]), torch.Size([5, 10]), torch.Size([500, 10]), torch.Size([1, 10]), torch.Size([5, 10]), torch.Size([500, 10]), torch.Size([1, 10]), torch.Size([5, 10]), torch.Size([500, 10]), torch.Size([1, 10]), torch.Size([5, 10]), torch.Size([500, 10]), torch.Size([1, 1000]), torch.Size([5, 1000]), torch.Size([500, 1000]), torch.Size([1, 1000]), torch.Size([5, 1000]), torch.Size([500, 1000]), torch.Size([1, 1000]), torch.Size([5, 1000]), torch.Size([500, 1000]), torch.Size([1, 1000]), torch.Size([5, 1000]), torch.Size([500, 1000])]

    sanityCheckModel(cbow_forward_inputs, CbowModel, cbow_forward_expected_outputs, "forward", makeCbowSanityBatch)

--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 2}	Expected Num. Params: 3082	Your Num. Params: 3082
	PASSED	Input: {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 64, 'context_size': 4}	Expected Num. Params: 3082	Your Num. Params: 3082
	PASSED	Input: {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 128, 'context_size': 2}	Expected Num. Params: 5834	Your Num. Params: 5834
	PASSED	Input: {'vocab_size': 10, 'embed_size': 32, 'hidden_size': 128, 'context_size': 4}	Expected Num. Params: 5834	Your Num. Params: 5834
	PASSED	Input: {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 2}	Expected Num. Params: 5450	Your Num. Params: 5450
	PASSED	Input: {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 64, 'context_size': 4}	Expected Num. Params: 5450	Your Num. Params: 5450
	PASSED	Input: {'vocab_size': 10, 'embed_size': 64, 'hidden_size': 128, 'context_size': 2}	Expe

## Train the CBOW Model

In [10]:
# A dataloader is responsible for providing batches of data to your model. Notice how we first instantiate dataset.

BATCH_SIZE = 1000 # You may change the batch size if you'd like
CONTEXT_SIZE = 3  # You may change the context size if you'd like

if __name__=='__main__':
    cbow_dataset = CbowDataset(sentences, vocab, CONTEXT_SIZE)
    cbow_dataloader = torch.utils.data.DataLoader(cbow_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=True)

In [11]:
from tqdm.notebook import tqdm
from torch import optim

def train_cbow_model(model, num_epochs, data_loader, optimizer, criterion):
    print("Training CBOW model....")
    for epoch in range(num_epochs):
        epoch_loss, n = 0, 0
        for context, target in tqdm(data_loader):
            optimizer.zero_grad()
            log_probs = model(context.long().to(DEVICE)) # to(torch.float32)
            loss = criterion(log_probs, target.to(DEVICE))
            loss.backward()
            optimizer.step()
            n += context.shape[0]
            epoch_loss += (loss*context.shape[0])

        epoch_loss = epoch_loss/n
        print('[TRAIN]\t Epoch: {:2d}\t Loss: {:.4f}'.format(epoch+1, epoch_loss))
    print('CBOW Model Trained!\n')

Initialize the model.

In [12]:
if __name__=='__main__':
    cbow_model = CbowModel(vocab_size = cbow_dataset.vocab_size,
                embed_size = 128,
                hidden_size = 128,
                context_size = CONTEXT_SIZE)

    cbow_model = cbow_model.to(DEVICE)

    print('The model has {:,d} trainable parameters'.format(count_parameters(cbow_model)))

The model has 7,438,415 trainable parameters


Create the criterion to measure the loss of the model.

In [13]:
import torch.optim as optim

if __name__=='__main__':
    LEARNING_RATE = 0.01

    # Define the loss function
    criterion = nn.CrossEntropyLoss().to(DEVICE)

    # Define the optimizer
    optimizer = optim.Adam(cbow_model.parameters(), lr=LEARNING_RATE)

In [14]:
if __name__=='__main__':
    N_EPOCHS = 6 # Feel free to change this

    # Train model for N_EPOCHS epochs
    train_cbow_model(cbow_model, N_EPOCHS, cbow_dataloader, optimizer, criterion)

Training CBOW model....


  0%|          | 0/1684 [00:00<?, ?it/s]

[TRAIN]	 Epoch:  1	 Loss: 6.8533


  0%|          | 0/1684 [00:00<?, ?it/s]

[TRAIN]	 Epoch:  2	 Loss: 6.2951


...

[TRAIN]	 Epoch:  4	 Loss: 5.8397


  0%|          | 0/1684 [00:00<?, ?it/s]

[TRAIN]	 Epoch:  5	 Loss: 5.7008


  0%|          | 0/1684 [00:00<?, ?it/s]

[TRAIN]	 Epoch:  6	 Loss: 5.5993
CBOW Model Trained!



Visualize word embeddings

In [15]:
if __name__=='__main__':
    from sklearn.manifold import TSNE
    import numpy as np
    import plotly.express as px
    import pandas as pd
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    THRESHOLD = 1000
    words = [x[0] for x in sorted(vocab.items(), key = lambda x: -x[1])[:THRESHOLD]]
    idxes = [cbow_dataset.word2idx[word] for word in words]
    vectors = np.array([cbow_model.embedding.weight[i].tolist() for i in idxes])

    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, verbose=False)
    new_vectors = tsne_model.fit_transform(vectors)

    df = pd.DataFrame(data={'x': new_vectors[:,0], 'y': new_vectors[:,1], 'word':words})

    fig = px.scatter(df, x='x', y='y', text='word')
    fig.update_traces(textposition='top center')
    fig.update_layout(height=600, title_text='Word Embedding 2D Visualization')
    fig.show()